In [1]:
import random
import re
import requests
import time
import math
import numpy as np
import operator
from bs4 import BeautifulSoup as bs
from collections import Counter
import os

In [2]:
def deleteBadWords(StrIn):
    Str_BadWords = u'延伸閱讀|連絡方式|電話預約|電話|營業時間|週一|週二|週三|週四|週五|週六|週日|周一|周二|周三|周四|周五|周六|\
                    |周日|假日|公休|平日|地址|粉絲團|星期|禮拜|時間限制|您或許對這些文章有興趣|造訪日期|全年無休|最後點餐|營業|AM|PM|上一篇|下一篇|\
                    |分享此文|您可能喜歡的文章|懶人包|臉書|Facebook|facebook|FB|fb|全世界便宜住宿看這兒|下載愛食記App隨時觀看|按個讚啦|喜歡我的分享嗎|\
                    |瘋台灣民宿網|官方網站|瀏覽人次|最新消息|餐廳名稱|消費時間|無圖文版|網誌|Postedonby|新鮮關注回聲|Christabelle的藝想世界部落格由製作|\
                    |也許對這些文章也有興趣|發表迴響|電子郵件|必要欄位標記|電子郵件|個人網站|輸入圖片顯示文字好證明你不是機器人|站內搜尋分類|最新動態|\
                    |並不會被公開|你的位址 |迴響名稱|用餐日期|留言|載入中|文章文章|粉絲頁|發表|每人平均價位|按個讚|推薦你閱讀|Instagram|instagram|\
                    |美食地圖|版權所有|網友回應|歡迎加入|標籤|著作權聲明|非經授權|不得轉載'
    strClean = re.sub(Str_BadWords,'',StrIn)
    return strClean

def EnglishFullToHalf(StrIn):
    def transform(ele):
        alphabetInt = int(repr(ele.group('number'))[4:8],16)
        transAlphabeInt = alphabetInt - 65248
        return binascii.a2b_hex(hex(transAlphabeInt)[2:4])
    pattern = re.sub(u'(?P<number>[\uff21-\uff3a\uff41-\uff5a])', transform, StrIn)
    return pattern
                
def retain_English_Chinese_Arabic_numerals(StrIn):
    Str_English_Chinese = u'([^ 0-9a-zA-Zａ-ｚＡ-Ｚ\u4E00-\u9FCC]+)'
    #Str_English_Chinese = u'([^a-z^A-Z^ａ-ｚ^Ａ-Ｚ^^0-9^０-９^\u3105-\u3129^\u4E00-\u9FCC]+)'
    #\u3105-\u3129為所有注音符號 
    #\u4E00-\u9FCC為所有中文
    strClean = re.sub(Str_English_Chinese,'',StrIn)
    return strClean

def removeSentimentInAds(StrIn):
    pattern = u'.*(喜歡|推薦|喜愛).{0,6}(文章|本文|介紹)'
    def sub(match):
        string = match.group(0)
        type1 = match.group(1) if match.group(1) else ''
        r = string.replace(type1,'')
        return r
    result = re.sub(pattern, sub, StrIn)
    return result
def removeTag(soup,tag):
    [x.extract() for x in soup.select(tag)]

brand=['HTC','Sony','Asus','Acer','Samsung','LG','Motorola','InFocus','GSmart','OPPO','TWM','OKWAP','HUAWEI']
model=['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P','Q','R','S','T','U','V','W','X','Y','Z']
rs = requests.session()
rs.headers.update({
        'Host':'www.mobile01.com',
        "User-Agent": "Mozilla/5.0 (Linux; Android 4.{0}.{1};{2} {3}{4}-{5}{6}) \
        AppleWebKit/537.36 (KHTML, like Gecko) Version/{7}.0 Chrome/30.0.0.0 Mobile Safari/537.36"\
        .format(random.randint(0,9),random.randint(0,9),brand[random.randint(0,len(brand)-1)],\
        model[random.randint(0,len(model)-1)],random.randint(1,99),model[random.randint(0,len(model)-1)],\
        random.randint(799,1599),random.randint(250,9999))
    })

In [17]:
import requests
import re
import random
import time
from bs4 import BeautifulSoup as bs

def geturl(f, p, rs):
    urls = []
    domain = 'http://www.mobile01.com/'
    url = 'http://www.mobile01.com/waypointtopiclist.php?f={}&p={}'
    #url = 'http://www.mobile01.com/waypointforumtopic.php?p={}'
    res = rs.get(url.format(f,p))
    soup = bs(res.text,'html.parser')
    page = int(re.search('共(.+)頁', soup.select('.numbers')[0].encode('utf-8')).group(1))
    titles = soup.select('.subject-text')
    for t in titles:
        urls.append(domain+t.select('a')[0]['href'])
    return urls

def getFinalPage(url, rs):
    res = rs.get(url)
    soup = bs(res.text,'lxml')
    page = int(re.search('共(.+)頁', soup.select('.numbers')[0].encode('utf-8')).group(1))
    return page

def getcontent(url, rs):
    res = rs.get(url)
    soup = bs(res.text,'html.parser')
    removeTag(soup,'script')
    removeTag(soup,'a')
    removeTag(soup,'.rank')
    removeTag(soup,'iframe')
    removeTag(soup,'.fsb-social-bar')
    removeTag(soup,'small')
    removeTag(soup,'.comment-content.comment')
    removeTag(soup,'.moreincategories.clearfix')
    removeTag(soup,'.relativepost.clearfix')
    removeTag(soup,'.auth')
    removeTag(soup,'.store')
    removeTag(soup,'.comments-area')
    removeTag(soup,'.sharedaddy.sd-sharing-enabled')
    removeTag(soup,'.vcard')
    removeTag(soup,'#facebook')
    removeTag(soup,'#sidebar')
    removeTag(soup,'#jp-relatedposts')
    removeTag(soup,'.hot-info')
    removeTag(soup,'.agoda-link')
    removeTag(soup,'#notice')
    removeTag(soup,'.tag')
    removeTag(soup,'blockquote')
    t = soup.select('.single-post-content')[0]
    s =  "".join("".join(t.text).split())
    return s

def getall(f, p):
    st = []
    ul = geturl(f, p, rs)
    for u in ul:
        try:
            st.append(getcontent(u, rs))
        except:
            print u
        time.sleep(0.5)
    return st

In [24]:
#import autoDict
# 一整頁文章一起跑
#頁數範圍
for f in xrange(196,213):
    url = 'http://www.mobile01.com/waypointtopiclist.php?f={}'
    try:
        fp = getFinalPage(url.format(f), rs)
    except IndexError:
        continue
    for p in xrange(1,fp+1):
        with open('./Workspace/data/mobile01/article_%s_%d.txt' %(f,p), 'w') as fw:
            #獲取一整頁的文章
            news = getall(f, p)
            for a in news:
                fw.write(a.encode('utf-8')+'\n')
            #autoDict.getwords(news)

In [5]:
import codecs
temp = set()
for fname in os.listdir('./Workspace/data/dictionary'):
    if 'txt' in fname:
        with codecs.open('./Workspace/data/dictionary/%s' %fname, 'r', 'utf-8') as f:
            for t in f.readlines():
                if not re.search('\w', t) and u'的' not in t:
                    temp.add(t.strip('\n'))
with open('./Workspace/data/dictionary/all/dictionary.txt', 'w') as f:
    for x in list(temp):
        f.write(x.encode('utf-8')+'\n')
print len(temp)

6499
